# Реализуем методы для наивного байеса

Сгенерируем выборку, в которой каждый признак имеет некоторое своё распределение, параметры которого отличаются для каждого класса. Затем реализуем несколько методов для класса, который уже частично написан ниже:
- метод predict
- метод \_find\_expon\_params и \_get\_expon\_density для экспоненциального распределения
- метод \_find\_norm\_params и \_get\_norm\_probability для биномиального распределения

Для имплементации \_find\_something\_params изучите документацию функций для работы с этими распределениями в scipy.stats и используйте предоставленные там методы.

In [1]:
import numpy as np
import scipy
import scipy.stats

Сформируем параметры генерации для трех датасетов

In [2]:
func_params_set0 = [(scipy.stats.bernoulli, [dict(p=0.1), dict(p=0.5)]),
                   ]

func_params_set1 = [(scipy.stats.bernoulli, [dict(p=0.1), dict(p=0.5)]),
                    (scipy.stats.expon, [dict(scale=1), dict(scale=0.3)]),
                   ]

func_params_set2 = [(scipy.stats.bernoulli, [dict(p=0.1), dict(p=0.5)]),
                    (scipy.stats.expon, [dict(scale=1), dict(scale=0.3)]),
                    (scipy.stats.norm, [dict(loc=0, scale=1), dict(loc=1, scale=2)]),
                   ]

def generate_dataset_for_nb(func_params_set=[], size = 2500, random_seed=0):
    '''
    Генерирует выборку с заданными параметрами распределений P(x|y).
    Число классов задается длиной списка с параметрами.
    Возвращает X, y, список с названиями распределений
    '''
    np.random.seed(random_seed)

    X = []
    names = []
    for func, params in func_params_set:
        names.append(func.name)
        f = []
        for i, param in enumerate(params):
            f.append(func.rvs(size=size, **param))
        f = np.concatenate(f).reshape(-1,1)
        X.append(f)

    X = np.concatenate(X, 1)
    y = np.array([0] * size + [1] * size)

    shuffle_inds = np.random.choice(range(len(X)), size=len(X), replace=False)
    X = X[shuffle_inds]
    y = y[shuffle_inds]

    return X, y, names 

X, y, distrubution_names = generate_dataset_for_nb(func_params_set0)
X.shape, y.shape, distrubution_names

((5000, 1), (5000,), ['bernoulli'])

In [3]:
X

array([[0],
       [1],
       [0],
       ...,
       [0],
       [1],
       [0]])

In [4]:
y

array([0, 1, 1, ..., 1, 1, 1])

In [22]:
from collections import defaultdict
from sklearn.base import BaseEstimator, ClassifierMixin

class NaiveBayes(BaseEstimator, ClassifierMixin):
    '''
    Реализация наивного байеса, которая помимо X, y
    принимает на вход во время обучения 
    виды распределений значений признаков
    '''
    def __init__(self):
        pass
    
    def _find_bernoulli_params(self, x):
        '''
        метод возвращает найденный параметр `p`
        распределения scipy.stats.bernoulli
        '''
        return dict(p=np.mean(x))
    
    def _get_bernoulli_probability(self, x, params):
        '''
        метод возвращает вероятность x для данных
        параметров распределния
        '''
        return scipy.stats.bernoulli.pmf(x, **params)

    def _find_expon_params(self, x):
        # нужно определить параметры распределения
        # и вернуть их
        loc, scale = scipy.stats.expon.fit(x, floc=0)
        return dict(scale=scale)
    
    def _get_expon_density(self, x, params):
        # нужно вернуть плотность распределения в x
        return scipy.stats.expon.pdf(x, **params)

    def _find_norm_params(self, x):
        # нужно определить параметры распределения
        # и вернуть их
        loc, scale = scipy.stats.norm.fit(x)
        return dict(loc=loc, scale=scale)
    
    def _get_norm_density(self, x, params):
        # нужно вернуть плотность распределения в x
        return scipy.stats.norm.pdf(x, **params)

    def _get_params(self, x, distribution):
        '''
        x - значения из распределения,
        distribution - название распределения в scipy.stats
        '''
        if distribution == 'bernoulli':
            return self._find_bernoulli_params(x)
        elif distribution == 'expon':
            return self._find_expon_params(x)
        elif distribution == 'norm':
            return self._find_norm_params(x)
        else:
            raise NotImplementedError('Unknown distribution')
            
    def _get_probability_or_density(self, x, distribution, params):
        '''
        x - значения,
        distribution - название распределения в scipy.stats,
        params - параметры распределения
        '''
        if distribution == 'bernoulli':
            return self._get_bernoulli_probability(x, params)
        elif distribution == 'expon':
            return self._get_expon_density(x, params)
        elif distribution == 'norm':
            return self._get_norm_density(x, params)
        else:
            raise NotImplementedError('Unknown distribution')

    def fit(self, X, y, distrubution_names):
        '''
        X - обучающая выборка,
        y - целевая переменная,
        feature_distributions - список названий распределений, 
        по которым предположительно распределны значения P(x|y)
        ''' 
        assert X.shape[1] == len(distrubution_names)
        assert set(y) == {0, 1}
        self.n_classes = len(np.unique(y))
        self.distrubution_names = distrubution_names
        
        self.y_prior = [(y == j).mean() for j in range(self.n_classes)]
        
        self.y = y
        self.distributions_params = defaultdict(dict)
        for i in range(X.shape[1]):
            distribution = self.distrubution_names[i]
            for j in range(self.n_classes):
                values = X[y == j, i]
                self.distributions_params[j][i] = \
                    self._get_params(values, distribution)
        
        return self.distributions_params
    
    def get_p_x_c(self, x, c, feature):
        #сколько всего меток класса с было
        y = self.y
        number_c = self.number_c_1
        if (c == 0):
            number_c = self.number_c_0
        
        number_x_and_c = 0
        for i in range(len(feature)):
            if (feature[i] == x and y[i] == c):
                number_x_and_c += 1
        return number_x_and_c / number_c
    
    def predict(self, X):
        '''
        X - тестовая выборка
        '''
        assert X.shape[1] == len(self.distrubution_names)
        
        # нужно реализовать подсчет аргмаксной формулы, по которой 
        # наивный байес принимает решение о принадлежности объекта классу
        # и применить её для каждого объекта в X
        #
        # примечание: обычно подсчет этой формулы реализуют через 
        # её логарифмирование, то есть, через сумму логарифмов вероятностей, 
        # поскольку перемножение достаточно малых вероятностей будет вести
        # к вычислительным неточностям
        
        probabl_classes_1 = []
        probabl_classes_0 = []
        
        y = self.y
        
        self.number_c_1 = len([y[i] for i in range(len(y)) if (y[i] == 1)])
        self.number_c_0 = len([y[i] for i in range(len(y)) if (y[i] == 0)])
        
        classes = np.unique(self.y_prior)
        for i in range(len(X[0])):
            cur_feature = list(X.T[i])
            distribution = self.distrubution_names[i]
            for j in range(len(cur_feature)):
                #предсказываем для класса 1, если вероятность будет меньше, то вернем класс 0
                x = cur_feature[j]
                
                #апостериорная вероятность класса (1\0) для данного признака
                p_x_c_1 = self.get_p_x_c(x, 1, cur_feature)
                p_x_c_0 = self.get_p_x_c(x, 0, cur_feature)
                
                params = self._get_params(x, distribution)
                p_x = self._get_probability_or_density(x, distribution, params)
                
                probabl_classes_1.append(np.log(p_x_c_1) - np.log(p_x))
                probabl_classes_0.append(np.log(p_x_c_0) - np.log(p_x))
            
            
        p_c_1 = self.number_c_1 / len(y)
        p_c_0 = self.number_c_0 / len(y)
        
        class_1 = np.reshape(np.array(probabl_classes_1), (len(X[0]), 5000))
        preds_1 = np.sum(class_1, axis=0) + np.log(p_c_1)
        
        class_0 = np.reshape(np.array(probabl_classes_0), (len(X[0]), 5000))
        preds_0 = np.sum(class_0, axis=0) + np.log(p_c_0)
        
        c = np.concatenate(([preds_0], [preds_1]), axis=0)
        preds = np.argmax(c, axis=0)
        return preds

In [27]:
a = np.array([[2, 1, 6, 8], [1, 1, 1, 1]])
np.sum(a, axis=0)
#np.argmax(a, axis = 0)

array([3, 2, 7, 9])

In [126]:
a = np.array([1, 2, 3])
b = np.array([4, 1, 8])
print(a)
print(b)
c = np.concatenate((a, b), axis=0)
c = np.reshape(c, (2, 3))
print(c)
np.argmax(c, axis=1)

[1 2 3]
[4 1 8]
[[1 2 3]
 [4 1 8]]


array([2, 2], dtype=int64)

In [132]:
k = np.sum(c, axis=0).T
k

array([ 5,  3, 11])

In [143]:
list(zip(list(k), list(k)))

[(5, 5), (3, 3), (11, 11)]

In [149]:
u = np.concatenate(([k], [k]), axis=0)
np.argmax(u, axis=0)

array([0, 0, 0], dtype=int64)

In [51]:
np.sum(c, axis=1)

array([ 5,  3, 11])

Проверим результат на примере первого распределения

In [23]:
nb = NaiveBayes()
nb.fit(X, y, ['bernoulli'])
#defaultdict(dict, {0: {0: {'p': 0.1128}}, 1: {0: {'p': 0.482}}})

defaultdict(dict, {0: {0: {'p': 0.1128}}, 1: {0: {'p': 0.482}}})

In [24]:
from sklearn.metrics import f1_score

prediction = nb.predict(X)
score = f1_score(y, prediction)
print('{:.4f}'.format(score))
#0.6045

class_1 = [[-0.65778004 -0.72981116 -0.65778004 ... -0.65778004 -0.72981116
  -0.65778004]]
sum = [-0.65778004 -0.72981116 -0.65778004 ... -0.65778004 -0.72981116
 -0.65778004]
len preds_1 = 5000
c = [[-0.81283202 -2.87528612 -0.81283202 ... -0.81283202 -2.87528612
  -0.81283202]
 [-1.35092722 -1.42295835 -1.35092722 ... -1.35092722 -1.42295835
  -1.35092722]]
len c = 2
len(preds) = 5000
0.6045


In [46]:
prediction

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

# Ответы для формы

Ответом для формы должны служить числа, которые будут выведены ниже. Все ответы проверены: в этих примерах получается одинаковый результат и через сумму логарифмов, и через произведение вероятностей.

In [25]:
scipy.stats.bernoulli.name

for fps in (func_params_set0 * 2,
            func_params_set1, 
            func_params_set2):
    

    X, y, distrubution_names = generate_dataset_for_nb(fps)
    print(f'X = {X}, y = {y}, distrubution_names = {distrubution_names}')
    nb = NaiveBayes()
    nb.fit(X, y, distrubution_names)
    prediction = nb.predict(X)
    #print(f'prediction = {prediction}')
    score = f1_score(y, prediction)
    print('{:.4f}'.format(score))
    
#0.7615

X = [[0 1]
 [0 0]
 [0 0]
 ...
 [0 1]
 [1 1]
 [0 0]], y = [1 0 0 ... 0 1 0], distrubution_names = ['bernoulli', 'bernoulli']
class_1 = [[-0.65778004 -0.65778004 -0.65778004 ... -0.65778004 -0.72981116
  -0.65778004]
 [-0.70360164 -0.68280089 -0.68280089 ... -0.70360164 -0.70360164
  -0.68280089]]
sum = [-1.36138168 -1.34058093 -1.34058093 ... -1.36138168 -1.4334128
 -1.34058093]
len preds_1 = 5000
c = [[-3.16041448 -0.91331556 -0.91331556 ... -3.16041448 -5.22286858
  -0.91331556]
 [-2.05452886 -2.03372811 -2.03372811 ... -2.05452886 -2.12655998
  -2.03372811]]
len c = 2
len(preds) = 5000
0.7615
X = [[0.         0.38970611]
 [0.         1.98976566]
 [0.         0.02420106]
 ...
 [0.         3.26481277]
 [1.         0.90650141]
 [0.         0.09442056]], y = [1 0 0 ... 0 1 0], distrubution_names = ['bernoulli', 'expon']


C:\Users\Nadezhda\Anaconda3\lib\site-packages\ipykernel_launcher.py:166: RuntimeWarning: divide by zero encountered in log
C:\Users\Nadezhda\Anaconda3\lib\site-packages\ipykernel_launcher.py:165: RuntimeWarning: divide by zero encountered in log


class_1 = [[-0.65778004 -0.65778004 -0.65778004 ... -0.65778004 -0.72981116
  -0.65778004]
 [-7.76640839        -inf        -inf ...        -inf -6.92220871
         -inf]]
sum = [-8.42418842        -inf        -inf ...        -inf -7.65201987
        -inf]
len preds_1 = 5000
c = [[        -inf  -6.94886116 -11.35823656 ...  -6.45367562         -inf
   -9.99687443]
 [ -9.1173356          -inf         -inf ...         -inf  -8.34516705
          -inf]]
len c = 2
len(preds) = 5000
1.0000
X = [[ 0.          0.07979793 -2.81833408]
 [ 1.          0.19636213  1.27899585]
 [ 1.          0.14832287 -0.60384868]
 ...
 [ 0.          0.8611044  -0.5600871 ]
 [ 0.          2.44201643  2.54627659]
 [ 0.          0.9297251   0.28562053]], y = [1 1 1 ... 0 1 0], distrubution_names = ['bernoulli', 'expon', 'norm']


C:\Users\Nadezhda\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1652: RuntimeWarning: invalid value encountered in double_scalars
  x = np.asarray((x - loc)/scale, dtype=dtyp)
C:\Users\Nadezhda\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:876: RuntimeWarning: invalid value encountered in greater_equal
  return (self.a <= x) & (x <= self.b)
C:\Users\Nadezhda\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:876: RuntimeWarning: invalid value encountered in less_equal
  return (self.a <= x) & (x <= self.b)


class_1 = [[-0.65778004 -0.72981116 -0.72981116 ... -0.65778004 -0.65778004
  -0.65778004]
 [-9.35230374 -8.45184071 -8.73240984 ...        -inf -5.93122191
         -inf]
 [        nan         nan         nan ...         nan         nan
          nan]]
sum = [nan nan nan ... nan nan nan]
len preds_1 = 5000
c = [[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]
len c = 2
len(preds) = 5000
0.0000


C:\Users\Nadezhda\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
